# Filtering Rows in Pandas

This article highlights various ways to filter rows in [Pandas](https://pandas.pydata.org/docs/user_guide/index.html#user-guide). The examples used here are based off the excellent [article](https://suzan.rbind.io/2018/02/dplyr-tutorial-3/) by [Susan Baert](https://twitter.com/SuzanBaert).

The data file can be accessed [here](https://github.com/samukweku/data_files/raw/master/msleep.txt)

## **Basic Row Filters**

In [1]:
import pandas as pd 
import numpy as np 
import janitor

In [2]:
file_path = "https://github.com/samukweku/data_files/raw/master/msleep.txt"

df = pd.read_csv(file_path)

df.head()

,name,genus,vore,order,conservation,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
0,Cheetah,Acinonyx,carni,Carnivora,lc,12.1,NaN,NaN,11.9,NaN,50.000
1,Owl monkey,Aotus,omni,Primates,NaN,17.0,1.8,NaN,7.0,0.01550,0.480
2,Mountain beaver,Aplodontia,herbi,Rodentia,nt,14.4,2.4,NaN,9.6,NaN,1.350
3,Greater short-tailed shrew,Blarina,omni,Soricomorpha,lc,14.9,2.3,0.133333,9.1,0.00029,0.019
4,Cow,Bos,herbi,Artiodactyla,domesticated,4.0,0.7,0.666667,20.0,0.42300,600.000


### Filtering Rows Based on a Numeric Variable

You can filter numeric variables based on their values. A number of commonly used operators include: >, >=, <, <=, == and !=.

In [3]:
df.loc[df.sleep_total > 18,  ["name", "sleep_total"]]

,name,sleep_total
21,Big brown bat,19.7
36,Thick-tailed opposum,19.4
42,Little brown bat,19.9
61,Giant armadillo,18.1


Another option is to use the [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method:

In [4]:
df.filter(["name", "sleep_total"]).query('sleep_total > 18')

,name,sleep_total
21,Big brown bat,19.7
36,Thick-tailed opposum,19.4
42,Little brown bat,19.9
61,Giant armadillo,18.1


Or [filter_on](https://pyjanitor.readthedocs.io/reference/janitor.functions/janitor.filter_on.html#janitor.filter_on), which is just a wrapper around the [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method,  from [pyjanitor](https://pyjanitor.readthedocs.io/):

In [5]:
df.select_columns(["name", "sleep_total"]).filter_on('sleep_total > 18')

,name,sleep_total
21,Big brown bat,19.7
36,Thick-tailed opposum,19.4
42,Little brown bat,19.9
61,Giant armadillo,18.1


```{note}
[loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) is significantly faster,especially when the number of rows are small
```

To select a range of values, you can use two logical requirements; in the example below, only rows where `sleep_total` is greater than or equal to 16, and less than or equal to 18 are selected:

In [6]:
df.loc[(df.sleep_total >= 16) & (df.sleep_total <= 18),  ["name", "sleep_total"]]

,name,sleep_total
1,Owl monkey,17.0
17,Long-nosed armadillo,17.4
19,North American Opossum,18.0
69,Arctic ground squirrel,16.6


An easier solution to the above is to use the [between](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.between.html) method:

In [7]:
df.loc[df.sleep_total.between(16, 18),  ["name", "sleep_total"]]


,name,sleep_total
1,Owl monkey,17.0
17,Long-nosed armadillo,17.4
19,North American Opossum,18.0
69,Arctic ground squirrel,16.6


The [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method offers a nice syntax for this as well:

In [8]:
df.filter(["name", "sleep_total"]).query('16 <= sleep_total <= 18')

,name,sleep_total
1,Owl monkey,17.0
17,Long-nosed armadillo,17.4
19,North American Opossum,18.0
69,Arctic ground squirrel,16.6


There are scenarios where you may want to select rows where the value is nearly a given value. You may also want to specify a tolerance value to indicate how far the values can be. [numpy.isclose](https://numpy.org/doc/stable/reference/generated/numpy.isclose.html) is a handy function for this:

Let's say the tolerance should be within one standard deviation of 17:

In [9]:
# calculate tolerance
tolerance = df['sleep_total'].std()

df.loc[np.isclose(df['sleep_total'], 17, atol = tolerance), ["name", "sleep_total"]]

,name,sleep_total
1,Owl monkey,17.0
2,Mountain beaver,14.4
3,Greater short-tailed shrew,14.9
5,Three-toed sloth,14.4
17,Long-nosed armadillo,17.4
19,North American Opossum,18.0
21,Big brown bat,19.7
26,Western american chipmunk,14.9
36,Thick-tailed opposum,19.4
38,Mongolian gerbil,14.2


### Filtering based on String Matches

You can select on string matches as well; in the example below, the ``==`` comparison operator is used to select a specific group of animals:

In [10]:
df.loc[df.order == "Didelphimorphia",  ["order", "name", "sleep_total"]]

,order,name,sleep_total
19,Didelphimorphia,North American Opossum,18.0
36,Didelphimorphia,Thick-tailed opposum,19.4


The [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method works here as well:

In [11]:
df.filter(["order", "name", "sleep_total"]).query('order == "Didelphimorphia"')

,order,name,sleep_total
19,Didelphimorphia,North American Opossum,18.0
36,Didelphimorphia,Thick-tailed opposum,19.4


Other operators can be used:

In [12]:
df.loc[df.order != "Rodentia",  ['order', 'name', 'sleep_total']]

,order,name,sleep_total
0,Carnivora,Cheetah,12.1
1,Primates,Owl monkey,17.0
3,Soricomorpha,Greater short-tailed shrew,14.9
4,Artiodactyla,Cow,4.0
5,Pilosa,Three-toed sloth,14.4
...,...,...,...
78,Scandentia,Tree shrew,8.9
79,Cetacea,Bottle-nosed dolphin,5.2
80,Carnivora,Genet,6.3
81,Carnivora,Arctic fox,12.5


In [13]:
df.loc[df['name'] > 'V',  ['order', 'name', 'sleep_total']]

,order,name,sleep_total
7,Rodentia,Vesper mouse,7.0
26,Rodentia,Western american chipmunk,14.9
40,Rodentia,Vole,12.8


In the examples above, only one animal is used; to select more animals, you could pass a list of conditions, with the `|` (or) symbol:

In [14]:
rows = (df.order == "Didelphimorphia") | (df.order == "Diprotodontia")

columns = ['order', 'name', 'sleep_total']

df.loc[rows, columns]

,order,name,sleep_total
19,Didelphimorphia,North American Opossum,18.0
36,Didelphimorphia,Thick-tailed opposum,19.4
57,Diprotodontia,Phalanger,13.7
60,Diprotodontia,Potoroo,11.1


However, this can become unwieldy, as the number of animals increase. The [isin](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.isin.html) method makes this easy:

In [15]:
df.loc[df.order.isin(["Didelphimorphia", "Diprotodontia"]), ["order", "name", "sleep_total"]]

,order,name,sleep_total
19,Didelphimorphia,North American Opossum,18.0
36,Didelphimorphia,Thick-tailed opposum,19.4
57,Diprotodontia,Phalanger,13.7
60,Diprotodontia,Potoroo,11.1


The [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method offers a nice syntax for this with python's [in](https://docs.python.org/3/reference/expressions.html#membership-test-operations) function:

In [16]:
(df.filter(["order", "name", "sleep_total"])
   .query('order in ["Didelphimorphia", "Diprotodontia"]')
)

,order,name,sleep_total
19,Didelphimorphia,North American Opossum,18.0
36,Didelphimorphia,Thick-tailed opposum,19.4
57,Diprotodontia,Phalanger,13.7
60,Diprotodontia,Potoroo,11.1


You could also use the ``==`` operator:

In [17]:
(df.filter(["order", "name", "sleep_total"])
   .query('order == ["Didelphimorphia", "Diprotodontia"]')
)

,order,name,sleep_total
19,Didelphimorphia,North American Opossum,18.0
36,Didelphimorphia,Thick-tailed opposum,19.4
57,Diprotodontia,Phalanger,13.7
60,Diprotodontia,Potoroo,11.1


You can also deselect certain groups using the [isin](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.isin.html) function, and combine it with the `~` symbol:

In [18]:
(df.loc[~df.order.isin(("Rodentia", "Carnivora", "Primates")),  ['order', 'name', 'sleep_total']]
   .head(10)
)

,order,name,sleep_total
3,Soricomorpha,Greater short-tailed shrew,14.9
4,Artiodactyla,Cow,4.0
5,Pilosa,Three-toed sloth,14.4
9,Artiodactyla,Roe deer,3.0
10,Artiodactyla,Goat,5.3
14,Soricomorpha,Star-nosed mole,10.3
16,Soricomorpha,Lesser short-tailed shrew,9.1
17,Cingulata,Long-nosed armadillo,17.4
18,Hyracoidea,Tree hyrax,5.3
19,Didelphimorphia,North American Opossum,18.0


With the [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method:

In [19]:
(df.filter(['order', 'name', 'sleep_total'])
   .query('order not in ("Rodentia", "Carnivora", "Primates")')
   .head(10)
)

,order,name,sleep_total
3,Soricomorpha,Greater short-tailed shrew,14.9
4,Artiodactyla,Cow,4.0
5,Pilosa,Three-toed sloth,14.4
9,Artiodactyla,Roe deer,3.0
10,Artiodactyla,Goat,5.3
14,Soricomorpha,Star-nosed mole,10.3
16,Soricomorpha,Lesser short-tailed shrew,9.1
17,Cingulata,Long-nosed armadillo,17.4
18,Hyracoidea,Tree hyrax,5.3
19,Didelphimorphia,North American Opossum,18.0


In [20]:
(df.filter(('order', 'name', 'sleep_total'))
   .query('order != ("Rodentia", "Carnivora", "Primates")')
   .head(10)
)

,order,name,sleep_total
3,Soricomorpha,Greater short-tailed shrew,14.9
4,Artiodactyla,Cow,4.0
5,Pilosa,Three-toed sloth,14.4
9,Artiodactyla,Roe deer,3.0
10,Artiodactyla,Goat,5.3
14,Soricomorpha,Star-nosed mole,10.3
16,Soricomorpha,Lesser short-tailed shrew,9.1
17,Cingulata,Long-nosed armadillo,17.4
18,Hyracoidea,Tree hyrax,5.3
19,Didelphimorphia,North American Opossum,18.0


The [filter_column_isin](https://pyjanitor.readthedocs.io/reference/janitor.functions/janitor.filter_column_isin.html#janitor.filter_column_isin) function from [pyjanitor](https://pyjanitor.readthedocs.io/index.html), which is just a wrapper around [isin](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.isin.html),  is an alternative:

In [21]:
(df.filter(('order', 'name', 'sleep_total'))
   .filter_column_isin(column_name = 'order', 
                       iterable = ("Rodentia", "Carnivora", "Primates"),
                       complement = True)
   .head(10)
)

,order,name,sleep_total
3,Soricomorpha,Greater short-tailed shrew,14.9
4,Artiodactyla,Cow,4.0
5,Pilosa,Three-toed sloth,14.4
9,Artiodactyla,Roe deer,3.0
10,Artiodactyla,Goat,5.3
14,Soricomorpha,Star-nosed mole,10.3
16,Soricomorpha,Lesser short-tailed shrew,9.1
17,Cingulata,Long-nosed armadillo,17.4
18,Hyracoidea,Tree hyrax,5.3
19,Didelphimorphia,North American Opossum,18.0


### Filtering Rows Based on Regex

There are scenarios where you need to filter string columns based on partial matches; Pandas has a wealth of [string methods](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#method-summary) that support regular expressions, and can be used in these situations.

Let's filter for rows where `mouse` can be found in the column `name`:

In [22]:
df.loc[df['name'].str.contains('mouse', case = False), ['name', 'sleep_total']]

,name,sleep_total
7,Vesper mouse,7.0
41,House mouse,12.5
46,Northern grasshopper mouse,14.5
56,Deer mouse,11.5
64,African striped mouse,8.7


```{margin} Note: Pyjanitor Function
[filter_string](https://pyjanitor.readthedocs.io/reference/janitor.functions/janitor.filter_string.html#janitor.filter_string) 
is a wrapper around the [Series.str.contains](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html#pandas.Series.str.contains) method, 

and can be handy in method chaining operations.
```

With the [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method:

In [23]:
df.filter(['name', 'sleep_total']).query('name.str.contains("mouse", case = False)', engine = 'python')

,name,sleep_total
7,Vesper mouse,7.0
41,House mouse,12.5
46,Northern grasshopper mouse,14.5
56,Deer mouse,11.5
64,African striped mouse,8.7


### Filtering Rows based on Multiple Conditions

Select rows with a `bodywt` above 100 and either have a `sleep_total` above 15, or are not part of the `Carnivora` `order`:

In [24]:
rows = (df.bodywt > 100) & ((df.sleep_total > 15) | (df.order != "Carnivora"))

columns = ["name", "order", slice("sleep_total", "bodywt")]

df.select_columns(columns).loc[rows]

,name,order,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
4,Cow,Artiodactyla,4.0,0.7,0.666667,20.00,0.423,600.000
20,Asian elephant,Proboscidea,3.9,NaN,NaN,20.10,4.603,2547.000
22,Horse,Perissodactyla,2.9,0.6,1.000000,21.10,0.655,521.000
23,Donkey,Perissodactyla,3.1,0.4,NaN,20.90,0.419,187.000
29,Giraffe,Artiodactyla,1.9,0.4,NaN,22.10,NaN,899.995
30,Pilot whale,Cetacea,2.7,0.1,NaN,21.35,NaN,800.000
35,African elephant,Proboscidea,3.3,NaN,NaN,20.70,5.712,6654.000
50,Tiger,Carnivora,15.8,NaN,NaN,8.20,NaN,162.564
76,Brazilian tapir,Perissodactyla,4.4,1.0,0.900000,19.60,0.169,207.501
79,Bottle-nosed dolphin,Cetacea,5.2,NaN,NaN,18.80,NaN,173.330


With the [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method:

In [25]:
(df.select_columns(["name", "order", slice("sleep_total", "bodywt")])
   .query('bodywt > 100 and (sleep_total > 15 or order != "Carnivora")')
)

,name,order,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
4,Cow,Artiodactyla,4.0,0.7,0.666667,20.00,0.423,600.000
20,Asian elephant,Proboscidea,3.9,NaN,NaN,20.10,4.603,2547.000
22,Horse,Perissodactyla,2.9,0.6,1.000000,21.10,0.655,521.000
23,Donkey,Perissodactyla,3.1,0.4,NaN,20.90,0.419,187.000
29,Giraffe,Artiodactyla,1.9,0.4,NaN,22.10,NaN,899.995
30,Pilot whale,Cetacea,2.7,0.1,NaN,21.35,NaN,800.000
35,African elephant,Proboscidea,3.3,NaN,NaN,20.70,5.712,6654.000
50,Tiger,Carnivora,15.8,NaN,NaN,8.20,NaN,162.564
76,Brazilian tapir,Perissodactyla,4.4,1.0,0.900000,19.60,0.169,207.501
79,Bottle-nosed dolphin,Cetacea,5.2,NaN,NaN,18.80,NaN,173.330


Return rows where `bodywt` is either greater than 100 or `brainwt` greater than 1, but not both:

In [26]:
rows = np.logical_xor((df.bodywt > 100), (df.brainwt > 1))

columns = ["name", "bodywt", "brainwt"]

df.loc[rows, columns]


,name,bodywt,brainwt
4,Cow,600.000,0.423
22,Horse,521.000,0.655
23,Donkey,187.000,0.419
29,Giraffe,899.995,NaN
30,Pilot whale,800.000,NaN
33,Human,62.000,1.320
50,Tiger,162.564,NaN
52,Lion,161.499,NaN
76,Brazilian tapir,207.501,0.169
79,Bottle-nosed dolphin,173.330,NaN


An alternative to the solution above, using the `!=` operator:

In [27]:
rows = (df.bodywt > 100) != (df.brainwt > 1)

columns = ["name", "bodywt", "brainwt"]

df.loc[rows, columns]

,name,bodywt,brainwt
4,Cow,600.000,0.423
22,Horse,521.000,0.655
23,Donkey,187.000,0.419
29,Giraffe,899.995,NaN
30,Pilot whale,800.000,NaN
33,Human,62.000,1.320
50,Tiger,162.564,NaN
52,Lion,161.499,NaN
76,Brazilian tapir,207.501,0.169
79,Bottle-nosed dolphin,173.330,NaN


With the [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method:

In [28]:
(df.filter(["name", "bodywt", "brainwt"])
   .query('(bodywt > 100) != (brainwt > 1)')
)

,name,bodywt,brainwt
4,Cow,600.000,0.423
22,Horse,521.000,0.655
23,Donkey,187.000,0.419
29,Giraffe,899.995,NaN
30,Pilot whale,800.000,NaN
33,Human,62.000,1.320
50,Tiger,162.564,NaN
52,Lion,161.499,NaN
76,Brazilian tapir,207.501,0.169
79,Bottle-nosed dolphin,173.330,NaN


Select all rows where `brainwt` is larger than 1, but `bodywt` does not exceed 100:

In [29]:
rows = ~(df.bodywt > 100) & (df.brainwt > 1)

columns = ["name", "sleep_total", "brainwt", "bodywt"]

df.loc[rows, columns]

,name,sleep_total,brainwt,bodywt
33,Human,8.0,1.32,62.0


An alternative to the solution above:

In [30]:
rows = (df.bodywt <= 100) & (df.brainwt > 1)

columns = ["name", "sleep_total", "brainwt", "bodywt"]

df.loc[rows, columns]

,name,sleep_total,brainwt,bodywt
33,Human,8.0,1.32,62.0


In [31]:
(df.filter(["name", "sleep_total", "brainwt", "bodywt"])
   .query('bodywt <= 100 and brainwt > 1')
)

,name,sleep_total,brainwt,bodywt
33,Human,8.0,1.32,62.0


### Filtering out Empty Rows

Empty rows can be filtered out with the [notna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.notna.html#pandas.DataFrame.notna) function:

In [32]:
rows = df.conservation.notna()

columns = ["name", slice("conservation", "sleep_cycle")]

df.select_columns(columns).loc[rows].head(10)

,name,conservation,sleep_total,sleep_rem,sleep_cycle
0,Cheetah,lc,12.1,NaN,NaN
2,Mountain beaver,nt,14.4,2.4,NaN
3,Greater short-tailed shrew,lc,14.9,2.3,0.133333
4,Cow,domesticated,4.0,0.7,0.666667
6,Northern fur seal,vu,8.7,1.4,0.383333
8,Dog,domesticated,10.1,2.9,0.333333
9,Roe deer,lc,3.0,NaN,NaN
10,Goat,lc,5.3,0.6,NaN
11,Guinea pig,domesticated,9.4,0.8,0.216667
12,Grivet,lc,10.0,0.7,NaN


In [33]:
df.select_columns(columns).query('conservation.notna()', engine = 'python').head(10)

,name,conservation,sleep_total,sleep_rem,sleep_cycle
0,Cheetah,lc,12.1,NaN,NaN
2,Mountain beaver,nt,14.4,2.4,NaN
3,Greater short-tailed shrew,lc,14.9,2.3,0.133333
4,Cow,domesticated,4.0,0.7,0.666667
6,Northern fur seal,vu,8.7,1.4,0.383333
8,Dog,domesticated,10.1,2.9,0.333333
9,Roe deer,lc,3.0,NaN,NaN
10,Goat,lc,5.3,0.6,NaN
11,Guinea pig,domesticated,9.4,0.8,0.216667
12,Grivet,lc,10.0,0.7,NaN


## Filtering across Multiple Columns

### Filter across all Columns

It is possible to filter for rows based on values across columns.

Let's filter for rows across the selected columns, keeping only rows where any column has the pattern `Ca` inside:

In [34]:
(df.select_columns([slice('name', 'order'), 'sleep_total'])
   .drop(columns='vore')
   .loc[lambda df: df.select_dtypes('object')
                     .transform(lambda x: x.str.contains('Ca'))
                     .any(axis = 'columns')
        ]
)

,name,genus,order,sleep_total
0,Cheetah,Acinonyx,Carnivora,12.1
6,Northern fur seal,Callorhinus,Carnivora,8.7
7,Vesper mouse,Calomys,Rodentia,7.0
8,Dog,Canis,Carnivora,10.1
9,Roe deer,Capreolus,Artiodactyla,3.0
10,Goat,Capri,Artiodactyla,5.3
11,Guinea pig,Cavis,Rodentia,9.4
27,Domestic cat,Felis,Carnivora,12.5
31,Gray seal,Haliochoerus,Carnivora,6.2
50,Tiger,Panthera,Carnivora,15.8


The code above works great; however, we could abstract this further with a function:

In [35]:
def filter_rows(df, dtype, columns, condition, any_True = True):
    temp = df.copy()
    if dtype:
        temp = df.select_dtypes(dtype)
    if columns:
        booleans = temp.loc[:, columns].transform(condition)
    else:
        booleans = temp.transform(condition)
    if any_True:
        booleans = booleans.any(axis = 1)
    else:
        booleans = booleans.all(axis = 1)
        
    return df.loc[booleans]

In [36]:
(df.select_columns([slice('name', 'order'), 'sleep_total'])
   .drop(columns = 'vore')
   .pipe(filter_rows,
         dtype = 'object',
         columns = None,
         condition = lambda df: df.str.contains('Ca'),
         any_True = True
        )
)

,name,genus,order,sleep_total
0,Cheetah,Acinonyx,Carnivora,12.1
6,Northern fur seal,Callorhinus,Carnivora,8.7
7,Vesper mouse,Calomys,Rodentia,7.0
8,Dog,Canis,Carnivora,10.1
9,Roe deer,Capreolus,Artiodactyla,3.0
10,Goat,Capri,Artiodactyla,5.3
11,Guinea pig,Cavis,Rodentia,9.4
27,Domestic cat,Felis,Carnivora,12.5
31,Gray seal,Haliochoerus,Carnivora,6.2
50,Tiger,Panthera,Carnivora,15.8


Let's look at another example, to filter for rows, across selected columns, where any column has a value less than 0.1:

In [37]:
(df.select_columns(['name', slice('sleep_total', 'bodywt')])
   .pipe(filter_rows,
         dtype = 'number',
         columns = None,
         condition = lambda df: df < 0.1,
         any_True = True
        )
   .head(10)
)

,name,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
1,Owl monkey,17.0,1.8,NaN,7.0,0.01550,0.480
3,Greater short-tailed shrew,14.9,2.3,0.133333,9.1,0.00029,0.019
7,Vesper mouse,7.0,NaN,NaN,17.0,NaN,0.045
8,Dog,10.1,2.9,0.333333,13.9,0.07000,14.000
9,Roe deer,3.0,NaN,NaN,21.0,0.09820,14.800
11,Guinea pig,9.4,0.8,0.216667,14.6,0.00550,0.728
13,Chinchilla,12.5,1.5,0.116667,11.5,0.00640,0.420
14,Star-nosed mole,10.3,2.2,NaN,13.7,0.00100,0.060
15,African giant pouched rat,8.3,2.0,NaN,15.7,0.00660,1.000
16,Lesser short-tailed shrew,9.1,1.4,0.150000,14.9,0.00014,0.005


The above example only requires that at least one column has a value less than 0.1. What if the goal is to return only rows where all the columns have values above 1? 

In [38]:
(df.select_columns(['name', slice('sleep_total', 'bodywt')])
   .drop(columns = 'awake')
   .pipe(filter_rows,
         dtype = 'number',
         columns = None,
         condition = lambda df: df > 1,
         any_True = False
        )
)

,name,sleep_total,sleep_rem,sleep_cycle,brainwt,bodywt
33,Human,8.0,1.9,1.5,1.32,62.0


Return rows where the string columns contain null values:

In [39]:
(df.select_columns([slice('name', 'order'), slice('sleep_total', 'sleep_rem')])
   .pipe(filter_rows,
         dtype = "object",
         columns = None,
         condition = lambda df: df.isna(),
         any_True = True
        )
)

,name,genus,vore,order,sleep_total,sleep_rem
7,Vesper mouse,Calomys,NaN,Rodentia,7.0,NaN
54,Desert hedgehog,Paraechinus,NaN,Erinaceomorpha,10.3,2.7
56,Deer mouse,Peromyscus,NaN,Rodentia,11.5,NaN
57,Phalanger,Phalanger,NaN,Diprotodontia,13.7,1.8
62,Rock hyrax,Procavia,NaN,Hyracoidea,5.4,0.5
68,Mole rat,Spalax,NaN,Rodentia,10.6,2.4
72,Musk shrew,Suncus,NaN,Soricomorpha,12.8,2.0


### Filter at

It is also possible to filter rows based on specific columns:

In [40]:
(df.select_columns(['name', 
                    slice('sleep_total', 'sleep_rem'), 
                    slice('brainwt', 'bodywt')]
                    )
   .loc[lambda df: df.filter(['sleep_rem', 'sleep_total'])
                     .gt(5)
                     .all(axis = 'columns')
        ]
)

,name,sleep_total,sleep_rem,brainwt,bodywt
36,Thick-tailed opposum,19.4,6.6,NaN,0.37
61,Giant armadillo,18.1,6.1,0.081,60.00


With the [query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) method:

In [41]:
(df.select_columns(['name', 
                    slice('sleep_total', 'sleep_rem'), 
                    slice('brainwt', 'bodywt')]
                    )
   .query('sleep_total > 5 and sleep_rem > 5')
)

,name,sleep_total,sleep_rem,brainwt,bodywt
36,Thick-tailed opposum,19.4,6.6,NaN,0.37
61,Giant armadillo,18.1,6.1,0.081,60.00


Or, using the function created earlier:

In [42]:
(df.select_columns(['name', 
                    slice('sleep_total', 'sleep_rem'), 
                    slice('brainwt', 'bodywt')]
                    )
   .pipe(filter_rows,
         dtype = None,
         columns = ['sleep_total', 'sleep_rem'],
         condition = lambda df: df > 5,
         any_True = False
        )
)

,name,sleep_total,sleep_rem,brainwt,bodywt
36,Thick-tailed opposum,19.4,6.6,NaN,0.37
61,Giant armadillo,18.1,6.1,0.081,60.00


Another example below that uses a different option when selecting the columns to filter at:

In [43]:
(df.select_columns(['name', 
                    slice('sleep_total', 'sleep_rem'), 
                    slice('brainwt', 'bodywt')]
                    )
   .loc[lambda df: df.filter(like='sleep')
                     .gt(5)
                     .all( axis = 'columns')
        ]
)

,name,sleep_total,sleep_rem,brainwt,bodywt
36,Thick-tailed opposum,19.4,6.6,NaN,0.37
61,Giant armadillo,18.1,6.1,0.081,60.00


In [44]:
(df.select_columns(['name', 
                    slice('sleep_total', 'sleep_rem'), 
                    slice('brainwt', 'bodywt')]
                    )
   .pipe(filter_rows,
         dtype = None,
         columns = lambda df: df.columns.str.contains('sleep'),
         condition = lambda df: df > 5,
         any_True = False
        )
)

,name,sleep_total,sleep_rem,brainwt,bodywt
36,Thick-tailed opposum,19.4,6.6,NaN,0.37
61,Giant armadillo,18.1,6.1,0.081,60.00


Resources: 

- [pandas docs](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) -  version ``1.2.3``
- [pyjanitor functions](https://pyjanitor.readthedocs.io/reference/general_functions.html) - version ``0.20.13``
- [numpy](https://numpy.org/doc/stable/reference/generated/numpy.isclose.html) - version ``1.20.0``